In [1]:
%load_ext autoreload
%autoreload 2
import sys
import os
project_root = os.path.abspath("")
if project_root not in sys.path:
    sys.path.append(project_root)
import sleap
from pathlib import Path
from ipywidgets import widgets
from IPython.display import display

%matplotlib widget

In [3]:
import sleap
import pandas as pd
from pathlib import Path

video_number = 3
slp_path = Path("/Users/joschua/repos/harris_lab/sleap-hypnose/predictions/sub_38_20251027_video3.v001.slp.251031_175928.predictions.slp")
output_path = Path(f"/Volumes/harris/hypnose/derivatives/sub-038_id-235/ses-008_date-20251027/saved_analysis_results/sleap_tracking_video{video_number}.csv")
output_path.parent.mkdir(parents=True, exist_ok=True)

# Load predictions
labels = sleap.load_file(str(slp_path))

# Extract all node positions for all frames
data = []
for lf in labels:
    frame_idx = lf.frame_idx
    for instance_idx, instance in enumerate(lf.instances):
        row = {"frame": frame_idx, "instance": instance_idx}
        for node, point in instance.nodes_points:
            row[f"{node.name}_x"] = point.x
            row[f"{node.name}_y"] = point.y
            row[f"{node.name}_score"] = point.score
        data.append(row)

df = pd.DataFrame(data)

# Calculate centroid from core nodes
core_nodes = ['right_ear', 'left_ear', 'center_head', 'neck', 'center', 'center_back']
centroid_x_cols = [f"{node}_x" for node in core_nodes]
centroid_y_cols = [f"{node}_y" for node in core_nodes]
centroid_score_cols = [f"{node}_score" for node in core_nodes]

df['centroid_x'] = df[centroid_x_cols].mean(axis=1)
df['centroid_y'] = df[centroid_y_cols].mean(axis=1)
df['centroid_score'] = df[centroid_score_cols].mean(axis=1)

# Save to CSV
df.to_csv(output_path, index=False)

# Summary output
print(f"Saved to: {output_path}")
print(f"\nTotal frames processed: {len(df)}")
print(f"Frame index range: {int(df['frame'].min())} - {int(df['frame'].max())}")
print(f"Frames with valid centroid: {df['centroid_x'].notna().sum()}")
print(f"\nCentroid score range: {df['centroid_score'].min():.4f} - {df['centroid_score'].max():.4f}")


Saved to: /Volumes/harris/hypnose/derivatives/sub-038_id-235/ses-008_date-20251027/saved_analysis_results/sleap_tracking_video3.csv

Total frames processed: 17114
Frame index range: 0 - 18473
Frames with valid centroid: 17114

Centroid score range: 0.2036 - 1.1391
